In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Enumerate all possible N=2 random walks

In [ ]:
directions = [(1,0), (0,1), (-1,0), (0,-1)]

In [18]:
rows = []
for dx1, dy1 in directions:
    for dx2, dy2 in directions:
        x = dx1 + dx2
        y = dy1 + dy2
        rows.append({'x': x, 'y': y})
df = pd.DataFrame(rows)

In [19]:
df

x  y
0   2  0
1   1  1
2   0  0
3   1 -1
4   1  1
5   0  2
6  -1  1
7   0  0
8   0  0
9  -1  1
10 -2  0
11 -1 -1
12  1 -1
13  0  0
14 -1 -1
15  0 -2

## Enumerate all possible N=4 random walks

In [21]:
rows = []
for dx1, dy1 in directions:
    for dx2, dy2 in directions:
        for dx3, dy3 in directions:
            for dx4, dy4 in directions:
                x = dx1 + dx2 + dx3 + dx4
                y = dy1 + dy2 + dy3 + dy4
                rows.append({'x': x, 'y': y})
df = pd.DataFrame(rows)

In [22]:
df

x  y
0    4  0
1    3  1
2    2  0
3    3 -1
4    3  1
..  .. ..
251 -1 -3
252  1 -3
253  0 -2
254 -1 -3
255  0 -4

[256 rows x 2 columns]

This approach will not scale to larger values of $N$

Instead, consider each particular walk a path through a tree data structure.

<img width="40%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Tree_%28computer_science%29.svg/1200px-Tree_%28computer_science%29.svg.png" alt="Tree (computer science).svg">

<a href="https://commons.wikimedia.org/wiki/File:Tree_(computer_science).svg" title="via Wikimedia Commons">Paddy3118</a> 
[<a href="https://creativecommons.org/licenses/by-sa/4.0">CC BY-SA</a>]

Each node will have 4 children indicating a direction to take. The number of levels in the tree depends on the number of steps to take.

Traversing tree-like data structures are examined in computer science.  Here we review a technique called a *Depth First Transveral*. 

Here, each node represents the position (x, y) at a particular step along the random walk. Say we want to find all possible walks up to $N=4$.

In [111]:
N = 4

In [123]:
rootNode = {'n': 0, 'x': 0, 'y': 0}

In [124]:
directions = [(1,0), (0,1), (-1,0), (0,-1)]

In [129]:
def walk_tree(node):
    if node['n'] == N:
        yield node
        return

    for dx, dy in directions:
        newNode = {'x': node['x']+dx, 
                   'y': node['y']+dy, 
                   'n': node['n']+1}

        yield from walk_tree(newNode)

In [130]:
pd.DataFrame(walk_tree(rootNode))

x  y  n
0    4  0  4
1    3  1  4
2    2  0  4
3    3 -1  4
4    3  1  4
..  .. .. ..
251 -1 -3  4
252  1 -3  4
253  0 -2  4
254 -1 -3  4
255  0 -4  4

[256 rows x 3 columns]

Is recursion necessary? No, we could use a stack instead.

In [138]:
def walk_tree(rootNode):
    
    stack = [rootNode]
    while len(stack) > 0:
        node = stack.pop()
        
        if node['n'] == N:
            yield node
            continue
        
        for dx, dy in directions:
            newNode = {'x': node['x']+dx, 
                       'y': node['y']+dy, 
                       'n': node['n']+1 }
            stack.append(newNode)

In [140]:
pd.DataFrame(walk_tree(rootNode))

x  y  n
0    0 -4  4
1   -1 -3  4
2    0 -2  4
3    1 -3  4
4   -1 -3  4
..  .. .. ..
251  3  1  4
252  3 -1  4
253  2  0  4
254  3  1  4
255  4  0  4

[256 rows x 3 columns]

Another approach is to keep track of which direction we need to head next for each level. Here we are not tracking individual walks but only counting how many walks there are of each size.

In [204]:
def walk_tree(N=4):
    
    dir = np.zeros(N+1)
    count = np.zeros(N+1)
    
    n = 0
    
    while True:
        if dir[n] == 4:
            n = n - 1
            if n == 0:
                break
        else:
            count[n] += 1
            dir[n] += 1
            
            if n < N:
                n = n + 1
                dir[n] = 0
    return count

In [216]:
walk_tree(N=4)

array([  1.,   4.,  16.,  64., 256.])